In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(theme='ggplot',sharing='public',offline=True)
from IPython.display import display,HTML
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import roll_time_series


# Fix needed to pandas datareader
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import datetime

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
from xgboost import XGBRegressor

/Users/irochka/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/irochka/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/irochka/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/irochka/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/Users/irochka/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C 

/Users/irochka/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [2]:
pd.set_option('display.max_columns',10)
df = pd.read_excel('data/Metallolom.xls', 'Sheet2', parse_dates=True,index=['time'])
print(df.head(20))
print(df.columns)

                  time  price_1   name_2   name_4  name_5  name_6  name_7  \
0  2013-01-03 20:00:00   8400.0  10030.0   9450.0  7930.0  8110.0  8880.0   
1  2013-01-10 20:00:00   8520.0  10010.0   9510.0  7930.0  8070.0  9020.0   
2  2013-01-17 20:00:00   8440.0   9850.0   9470.0  7910.0  8230.0  8820.0   
3  2013-01-24 20:00:00   8390.0   9660.0   9400.0  7970.0  8360.0  8920.0   
4  2013-01-31 20:00:00   8220.0   9240.0   9420.0  7930.0  7640.0  8730.0   
5  2013-02-07 20:00:00   8410.0   9240.0   9400.0  8010.0  8630.0  8600.0   
6  2013-02-14 20:00:00   8340.0   9330.0   9330.0  8010.0  8630.0  8800.0   
7  2013-02-21 20:00:00   8250.0   9300.0   9340.0  7910.0  8610.0  8910.0   
8  2013-02-28 20:00:00   8270.0   9340.0   9430.0  7910.0  8500.0  8800.0   
9  2013-03-07 20:00:00   8400.0   9330.0   9400.0  9400.0  8710.0  8810.0   
10 2013-03-14 20:00:00   8360.0   9380.0   9480.0  9610.0  8710.0  8810.0   
11 2013-03-21 20:00:00   8370.0   9390.0   9500.0  9410.0  8730.0  8850.0   

In [3]:
df = df.drop(['price_1','name_2','name_4','name_5','name_6',
               'name_7','name_8','name_9'],axis=1)
df.head()

,time,Median
0,2013-01-03 20:00:00,8440.0
1,2013-01-10 20:00:00,8305.0
2,2013-01-17 20:00:00,8335.0
3,2013-01-24 20:00:00,8375.0
4,2013-01-31 20:00:00,8115.0


In [4]:
df_shift, y = make_forecasting_frame(df.Median, kind="price", max_timeshift=52, rolling_direction=1)

In [5]:
%%capture
X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
                     show_warnings=False)

 'value__fft_coefficient__coeff_27__attr_"angle"'
 'value__fft_coefficient__coeff_27__attr_"imag"'
 'value__fft_coefficient__coeff_27__attr_"real"'
 'value__fft_coefficient__coeff_28__attr_"abs"'
 'value__fft_coefficient__coeff_28__attr_"angle"'
 'value__fft_coefficient__coeff_28__attr_"imag"'
 'value__fft_coefficient__coeff_28__attr_"real"'
 'value__fft_coefficient__coeff_29__attr_"abs"'
 'value__fft_coefficient__coeff_29__attr_"angle"'
 'value__fft_coefficient__coeff_29__attr_"imag"'
 'value__fft_coefficient__coeff_29__attr_"real"'
 'value__fft_coefficient__coeff_30__attr_"abs"'
 'value__fft_coefficient__coeff_30__attr_"angle"'
 'value__fft_coefficient__coeff_30__attr_"imag"'
 'value__fft_coefficient__coeff_30__attr_"real"'
 'value__fft_coefficient__coeff_31__attr_"abs"'
 'value__fft_coefficient__coeff_31__attr_"angle"'
 'value__fft_coefficient__coeff_31__attr_"imag"'
 'value__fft_coefficient__coeff_31__attr_"real"'
 'value__fft_coefficient__coeff_32__attr_"abs"'
 'value__fft_coeffic

In [6]:
# drop constant features
print(X.shape)
X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
print(X.shape)

(373, 794)
(373, 429)


In [7]:
X_tr = X.iloc[:354]
X_test = X.iloc[354:]
y_tr = y.iloc[:354]
y_test = y.iloc[354:]
y_tr.tail()


350    14715.0
351    14530.0
352    14095.0
353    13775.0
354    13540.0
Name: value, dtype: float64

In [8]:
X_train = X_tr.iloc[:300]
X_val = X_tr.iloc[300:]
y_train = y_tr.iloc[:300]
y_val = y_tr.iloc[300:]

In [9]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [10]:
xgbr = XGBRegressor()
xgbr.fit(X_train,y_train)
print(xgbr)

[10:34:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


In [11]:
output_xgb = xgbr.predict(data = X_val)

In [12]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(output_xgb,y_val))

896.2974175347222


In [17]:
plt.figure(figsize=(18,8))
plt.plot(X_val.index(),y_val,color='blue')
plt.plot(X_val.index(),output_xgb,color='red')
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'index'

<Figure size 1296x576 with 0 Axes>

In [19]:
output2_xgb = xgbr.predict(X_test)

In [18]:
plt.figure(figsize=(18,8))
plt.plot(X_val.index,y_val,color='green')
#plt.plot(X.index,y_val,color='blue')
plt.plot(X_test.index,output2,color='red')
plt.savefig('TSFResh+XGBoost.png')
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'index'

<Figure size 1296x576 with 0 Axes>

In [30]:

xgbr.fit(X_tr,y_tr)
out_xgb = xgbr.predict(X_test)
out_xgb

[11:08:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([8067.344 , 8067.344 , 8054.5337, 8054.5337, 8067.344 , 8067.344 ,
       8067.344 , 8037.566 , 8037.566 , 8089.541 , 8037.566 , 8024.7554,
       8024.7554, 8048.762 , 8066.1914, 8090.1104, 8126.582 , 8109.4834,
       8100.5347], dtype=float32)

In [24]:
output2_xgb

array([14435.513, 14522.144, 14695.526, 14178.163, 14206.269, 14156.565,
       14005.974, 14028.88 , 14025.363, 13927.386, 13871.404, 13899.161,
       13824.635, 13833.1  , 13841.982, 13716.528, 14200.812, 14151.758,
       13952.275], dtype=float32)